In [68]:
import requests, re
from bs4 import BeautifulSoup
from IPython.display import HTML, display

import pandas as pd

# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss
# goes to page 278

# first page of thread:
# store data in lists --> pandas dataframe

# THREAD NAME
# <h3 class="post-title"><a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941">I can&#039;t live without my dad - anyone feel the same?</a></h3>

# THREAD ADDRESS
# page_link

# USERNAME
# <a href="/about-cancer/cancer-chat/users/bras1548" title="View user profile." class="username">bras1548</a>

# POST DATE
# <span class="post-created hidden-xs">22 Jun 2020 16:35</span>

# POST TEXT
# <div class="field field-name-field-harmony-text field-type-text-long field-label-hidden">

# POST IS A REPLY TO POST_ID
# <span class="post-is-reply-to">22 Jun 2020 21:04 in response to <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941" class="active">bras1548</a></span>

# POST ID
# <a name="post-1305941"></a>

#### FUNCTION: SCRAPE A THREAD PAGE

In [135]:
# FUNCTION: SCRAPE A THREAD PAGE
def page_scraper_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')
    
    # THREAD ADDRESS 
    thread_address_list = []
    thread_address = soup.find("link",{"rel":"canonical"})['href']
    thread_address = page_link
    #print(thread_address)

    # THREAD NAME
    thread_name_list = []
    thread_group = soup.find("h3", {"class": "post-title"})
    thread_name = thread_group.find("a").text
    #print(thread_name)

    # POST ID
    post_id_list = []
    post_id_group = soup.findAll("a", {"name": re.compile(r".*")})

    for post_id_g in post_id_group:
        if "post-" in post_id_g["name"]:
            post_id_list.append(post_id_g["name"])
            #print(post_id_g["name"])

    # POST USERNAME
    username_list = []
    user_group = soup.findAll("a", {"class": "username"})

    for user_g in user_group:    
        username = user_g.text
        username_list.append(username)

        # populate THREAD ADDRESS, THREAD NAME lists
        thread_address_list.append(thread_address)
        thread_name_list.append(thread_name)
        #print(username)

    # POST DATE
    date_list = []
    date_group = soup.findAll("span", {"class": "post-created hidden-xs"})

    for date_g in date_group:
        date = date_g.text
        date_list.append(date)
        #print(date)

    # POST TEXT
    text_list = []
    text_group = soup.findAll("div", {"class": "field field-name-field-harmony-text field-type-text-long field-label-hidden"})

    for text_g in text_group:
        text_tag = text_g.find("div").find("div")
        text = text_tag.text
        text_list.append(text)
        #print(text_tag.text, "\n")

    # POST IS A REPLY TO
    reply_to_list = []
    reply_group = soup.findAll("span", {"class": "post-is-reply-to"})

    # add "N/A only to first post in thread
    if len(reply_group) < len(text_group):
        reply_to_list.append("N/A")
    
    for reply_g in reply_group:
        reply_to_id = reply_g.find("a")['href']
        reply_to_id = "https://www.cancerresearchuk.org" + reply_to_id
        reply_to_id = reply_to_id.replace(thread_address + "#", "")
        reply_to_list.append(reply_to_id)
        #print(reply_to_id)

    # COLUMN LISTS INTO A LIST OF ROWS
    new_rows = []

    for j in range(len(thread_name_list)):
        row_list = [thread_name_list[j], thread_address_list[j], username_list[j],
                    post_id_list[j], date_list[j], text_list[j], reply_to_list[j]]

        new_rows.append(row_list)
    
    return new_rows



#### FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

In [162]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

# <li class="cr-pagination__next">
# <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1">Next</a></li>

def next_page_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')

    next_link = soup.find("li", {"class":"cr-pagination__next"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org/" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=4#post-1515206"))
print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"))



None
https://www.cancerresearchuk.org//about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1


In [129]:
# TEST PAGE SCRAPER FUNCTION

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=4"
new_rows = page_scraper_cancer_research(page_link)


In [130]:
for row in new_rows:
    print(row[0])
    print(row[1])
    print(row[2])
    print(row[3])
    print(row[4])
    print(row[5])
    print(row[6], "\n")

I can't live without my dad - anyone feel the same?
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same
bras1548
post-1514356
7 Feb 2021 13:08
Hi SamC93,Thanks for your thoughts. Like you, I find it extremely unhelpful when people say those things. Here are the things that I find completely useless:- "What would your father want for you?"- "He'll always be with you"- "You have to live your life now"- "It will get better with time"I don't WANT to live without my dad. He WAS my life. Or at least the key part of my life. He was the sunshine in my world. The idea of letting him go and living my own life (without him) is appalling to me. I'm glad you're finding strength somewhere 
post-1513781 

I can't live without my dad - anyone feel the same?
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same
gpb959
post-1514436
7 Feb 2021 14:46
Hi, I lost my mom 4 months ago and I 

#### FUNCTION: SCRAPE ONE THREAD

In [123]:
# FUNCTION: SCRAPE ONE THREAD

row_list = []
page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"

def thread_scraper_cancer_research(page_link):

    new_rows = page_scraper_cancer_research(page_link)
    next_link = next_page_cancer_research(page_link)
    
    if next_link is not None:
        new_rows.extend(thread_scraper_cancer_research(next_link))

    return new_rows

In [136]:
# TEST: SCRAPE ONE THREAD

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"

new_rows = thread_scraper_cancer_research(page_link)
print(len(new_rows))

63


#### FUNCTION: GET THREAD LINKS FROM PAGE

In [155]:
# FUNCTION: GET THREAD LINKS FROM PAGE

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss"

185.198.189.21

def thread_scraper_cancer_research(forum_link):

    thread_links = []
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

    # <div class="tl-cell views-field views-field-nothing" >
    # <a href="/about-cancer/cancer-chat/thread/has-anyone-tried-bereavement-counselling">

    thread_link_group = soup.find_all("div", {"class":"tl-cell views-field views-field-nothing"})
    
    for thread_link_gr in thread_link_group:    
        new_thread_link = thread_link_gr.find('a')['href']
        thread_links.append("https://www.cancerresearchuk.org" + new_thread_link)
        #print("https://www.cancerresearchuk.org" + new_thread_link, "\n")
        
    return thread_links
    


#### FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

In [165]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

def next_forum_page_cancer_research(forum_link):
    
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

# <li class="next last"><a title="Go to next page" href="
    next_link = soup.find("li", {"class":"next last"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss"))
print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=278"))



https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=1
None


#### FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

In [173]:
# FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

def thread_links_cancer_research(forum_link):
    
    thread_links = []
    current_page = forum_link
    new_links = thread_scraper_cancer_research(current_page)
    thread_links.extend(new_links)
    
    next_link = next_forum_page_cancer_research(current_page)
    #print(next_link)
    
    if next_link is not None:
        thread_links.extend(thread_links_cancer_research(next_link))
        
    return thread_links




In [177]:
# TEST FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss"
thread_links = thread_links_cancer_research(forum_link)


https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=1
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=2
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=3
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=4
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=5
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=6
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=7
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=8
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=9
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=10

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=83
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=84
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=85
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=86
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=87
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=88
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=89
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=90
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=91
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-los

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=164
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=165
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=166
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=167
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=168
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=169
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=170
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=171
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=172
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=245
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=246
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=247
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=248
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=249
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=250
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=251
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=252
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss?page=253
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping

In [ ]:
# RUN 'SCRAPE THREAD' ON THREAD_LINKS

coping_with_loss_data = []



In [181]:
# SAVE 'COPING WITH LOSS' FORUM LINKS TO FILE

#print(len(thread_links))

#for thread in thread_links:
#    print(thread)

cancer_research_coping_w_loss_links = pd.DataFrame(thread_links)
cancer_research_coping_w_loss_links.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_links.csv', sep ='\t', index = False, header=True)


In [137]:
# GLOBAL database for 
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_coping_w_loss = pd.DataFrame(new_rows, columns = column_names)  
print (cancer_research_coping_w_loss)

cancer_research_coping_w_loss.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_test.csv', sep ='\t', index = False, header=True)


                                          Thread_name  \
0   I can't live without my dad - anyone feel the ...   
1   I can't live without my dad - anyone feel the ...   
2   I can't live without my dad - anyone feel the ...   
3   I can't live without my dad - anyone feel the ...   
4   I can't live without my dad - anyone feel the ...   
5   I can't live without my dad - anyone feel the ...   
6   I can't live without my dad - anyone feel the ...   
7   I can't live without my dad - anyone feel the ...   
8   I can't live without my dad - anyone feel the ...   
9   I can't live without my dad - anyone feel the ...   
10  I can't live without my dad - anyone feel the ...   
11  I can't live without my dad - anyone feel the ...   
12  I can't live without my dad - anyone feel the ...   
13  I can't live without my dad - anyone feel the ...   
14  I can't live without my dad - anyone feel the ...   
15  I can't live without my dad - anyone feel the ...   
16  I can't live without my dad